# Введение. Полносвязные слои. Функции активации (ноутбук)

> Начнем осваивать библиотеку `PyTorch`.

## План ноутбука

1. Установка `PyTorch`
1. Введение в `PyTorch`
1. Полносвязные слои и функции активации в `PyTorch`
1. Градиентный спуск своими руками

## Установка `PyTorch`

Мы будем использовать библиотеку для глубинного обучения `PyTorch`, ее можно не устанавливать, можно пользоваться сайтами [Kaggle](kaggle.com) и [Google Colab](colab.research.google.com/) для обучения в облаке (или с учителем?). 

Чтобы установить `PyTorch` локально себе на компьютер нужно ответить на два вопроса - какая у вас операционная система и есть ли у вас дискретная видеокарта (GPU) и если есть, то какого производителя. В зависимости от ваших ответов мы получаем три варианта по операционной системе - Linux, Mac и Windows; три варианта по дискретной видеокарте - нет видеокарты (доступен только центральный процессор CPU), есть видеокарта от Nvidia или есть видеокарта от AMD (это производитель именно чипа, конечный вендор может быть другой, например, ASUS, MSI, Palit). Работа с PyTorch с видеокартой от AMD это экзотика, которая выходит за рамки нашего курса, поэтому рассмотрим только варианты *нет видеокарты*/*есть видеокарта от Nvidia*.


Выберите на [сайте](https://pytorch.org/get-started/locally/) подходящие вам варианты операционной системы/видеокарты и скопируйте команду для установки. Разберем подробно самые популярные варианты установки:

### Установка в Linux ([поддерживаемые дистрибутивы](https://pytorch.org/get-started/locally/#supported-linux-distributions))

На линуксе будет работать поддержка `PyTorch` в любой конфигурации, что у вас нет видеокарты, что есть от Nvidia, что от AMD. 

Пререквизит для работы с видеокартой от Nvidia - нужно поставить CUDA, это инструмент от компании Nvidia, который позволяет ускорять вычисления на их же ГПУ. Чтобы поставить себе на машину все правильно воспользуйтесь этим [гайдом](https://docs.nvidia.com/cuda/cuda-installation-guide-linux/index.html) от Nvidia.

 - **pip**

`pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu` для тех, у кого нет видеокарты.

`pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118` для тех, у кого есть видеокарта (либо другой `--extra-index-url`, смотрите на сайте PyTorch, в зависимости от версии CUDA).

 - **conda**

`conda install pytorch torchvision torchaudio cpuonly -c pytorch` для тех, у кого нет видеокарты.

`conda install pytorch torchvision torchaudio cudatoolkit=11.8 -c pytorch -c conda-forge` для тех, у кого есть видеокарта (либо немного другая команда, в зависимости от версии CUDA).

### Установка в Windows

На винде будет работать поддержка `PyTorch` только для видеокарт от Nvidia и без видеокарт вообще. 

Пререквизит для работы с видеокартой от Nvidia - нужно поставить CUDA, это инструмент от компании Nvidia, который позволяет ускорять вычисления на их же ГПУ. Чтобы поставить себе на машину все правильно воспользуйтесь этим [гайдом](https://docs.nvidia.com/cuda/cuda-installation-guide-microsoft-windows/index.html) от Nvidia.

 - **pip**

`pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu` для тех, у кого нет видеокарты.

`pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118` для тех, у кого есть видеокарта (либо другой `--extra-index-url`, смотрите на сайте PyTorch, в зависимости от версии CUDA).

 - **conda**

`conda install pytorch torchvision torchaudio cpuonly -c pytorch` для тех, у кого нет видеокарты.

`conda install pytorch torchvision torchaudio cudatoolkit=11.8 -c pytorch -c conda-forge` для тех, у кого есть видеокарта (либо немного другая команда, в зависимости от версии CUDA).



### Установка на Mac

На маках есть поддержка `PyTorch` как на CPU, так и на GPU, но только для Apple Silicon, то есть на чипах M1, M2 и так далее.

При этом поддержка ускорения с помощью GPU есть только для версий макоси выше 12.3: MPS acceleration is available on MacOS 12.3+

 - **pip**

`pip3 install torch torchvision torchaudio`

 - **conda**

`conda install pytorch::pytorch torchvision torchaudio -c pytorch`

## Введение в `PyTorch`

### Тензоры

Тензоры — это специализированная структура данных, по сути это массивы и матрицы. Тензоры очень похожи на массивы в numpy, так что, если у вас хорошо с numpy, то разобраться в PyTorch тензорах будет очень просто. В PyTorch мы используем тензоры для кодирования входных и выходных данных модели, а также параметров модели.

In [2]:
import torch
import numpy as np

### Создание тензоров

Тензор можно создать напрямую из каких-то данных - нам подходят все списки с числами:

In [3]:
some_data = [1, 2, 3, 4]
some_tensor = torch.tensor(some_data)

some_tensor

tensor([1, 2, 3, 4])

In [4]:
some_data = [[1, 2], [3, 4], [5, 6]]
some_tensor = torch.tensor(some_data)

some_tensor

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [5]:
some_data = [[[1], [2]], [[3], [4]], [[5], [6]]]
some_tensor = torch.tensor(some_data)

some_tensor

tensor([[[1],
         [2]],

        [[3],
         [4]],

        [[5],
         [6]]])

На самом деле про "все" списки с числами - обман. Если у вашего списка есть какой-то уровень вложенности, то должны совпадать размерности у всех вложенных списков (подробнее про размерности поговорим позже):

In [6]:
some_other_data = [[1, 2], [3, 4], [5, 6, 7]]
some_other_tensor = torch.tensor(some_other_data)

some_other_tensor

ValueError: expected sequence of length 2 at dim 1 (got 3)

Также тензоры можно создавать из numpy массивов и наоборот:

In [7]:
some_numpy_array = np.array(some_data)

some_numpy_array

array([[[1],
        [2]],

       [[3],
        [4]],

       [[5],
        [6]]])

In [8]:
some_tensor_from_numpy = torch.from_numpy(some_numpy_array)

some_tensor_from_numpy

tensor([[[1],
         [2]],

        [[3],
         [4]],

        [[5],
         [6]]])

При этом если мы создаем тензор из numpy массива с помощью `torch.from_numpy`, то они делят между собой память, где лежат их данные и, соответственно, при изменении тензора меняется numpy массив и наоборот:

In [9]:
x = np.ones(10)
y = torch.from_numpy(x)

x, y

(array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=torch.float64))

In [10]:
x += 1

x, y

(array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]),
 tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.], dtype=torch.float64))

In [11]:
x = torch.ones(10)
y = x.numpy()

x, y

(tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32))

In [12]:
x += 1

x, y

(tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]),
 array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.], dtype=float32))

Можем создать тензор со случайными или константными значениями:

In [13]:
shape = (2, 3)

random_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)
empty_tensor = torch.empty(shape)

random_tensor, ones_tensor, zeros_tensor, empty_tensor

(tensor([[0.7030, 0.8488, 0.0055],
         [0.3949, 0.7505, 0.5193]]),
 tensor([[1., 1., 1.],
         [1., 1., 1.]]),
 tensor([[0., 0., 0.],
         [0., 0., 0.]]),
 tensor([[8.9625e-01, 7.9309e+34, 7.9439e+08],
         [3.2604e-12, 7.3113e+34, 9.5492e-01]]))

Теперь поговорим про размерности подробнее.

У тензора есть какой-то размер, какая форма. Первое с чем нужно определиться, какой **размерности** тензор - количество осей у него.

In [14]:
shape = (10)  # одна ось (вектор)

tensor = torch.rand(shape)

tensor

tensor([0.7187, 0.2798, 0.0451, 0.6731, 0.1123, 0.6423, 0.7267, 0.3930, 0.6888,
        0.7260])

In [15]:
shape = (2, 3)  # две оси (матрица)

tensor = torch.rand(shape)

tensor

tensor([[0.8805, 0.4474, 0.4230],
        [0.7759, 0.6520, 0.4739]])

In [16]:
shape = (3, 2, 3)  # три оси (и больше - тензор)

tensor = torch.rand(shape)

tensor

tensor([[[0.1757, 0.9515, 0.0762],
         [0.6463, 0.0056, 0.4057]],

        [[0.7680, 0.1649, 0.8964],
         [0.1826, 0.3367, 0.3551]],

        [[0.3947, 0.0326, 0.9602],
         [0.5664, 0.8970, 0.2120]]])

Тензор с размерностью 1 - это просто вектор, список чисел.

Тензор с размерностью 2 - это просто матрица, то есть список списков чисел.

Тензор с размерностью 3 и больше - это тензор, то есть список списков списков ... чисел.

![](meme.png)

Получить доступ к размеру уже созданного тензора - метод `.shape`:

In [17]:
some_data = [[[1], [2]], [[3], [4]], [[5], [6]]]
some_tensor = torch.tensor(some_data)

print(some_tensor)
print(some_tensor.shape)

tensor([[[1],
         [2]],

        [[3],
         [4]],

        [[5],
         [6]]])
torch.Size([3, 2, 1])


Давайте сделаем тензор, который будет нам имитировать изображение - сделаем его размер `(c, h, w)`, где `h` и `w` это его высота и ширина, а `c` - число каналов в цветовом пространстве (в черно-белом 1, в RGB 3):

In [18]:
h = 9
w = 16
c = 3

shape = (c, h, w)

image_tensor = torch.rand(shape)

image_tensor

tensor([[[0.4864, 0.1345, 0.0985, 0.4910, 0.5191, 0.5402, 0.9424, 0.2336,
          0.8935, 0.6876, 0.0369, 0.8935, 0.2823, 0.6079, 0.8713, 0.8276],
         [0.4903, 0.8007, 0.8147, 0.8142, 0.5783, 0.7870, 0.6481, 0.8210,
          0.4620, 0.7126, 0.3961, 0.2064, 0.6209, 0.7252, 0.0013, 0.8203],
         [0.6782, 0.4787, 0.7272, 0.7661, 0.1971, 0.1114, 0.1537, 0.6295,
          0.8935, 0.4559, 0.6645, 0.7148, 0.9582, 0.4237, 0.3216, 0.0474],
         [0.9153, 0.7582, 0.9807, 0.7008, 0.4528, 0.5591, 0.9300, 0.8818,
          0.0474, 0.9770, 0.0913, 0.7871, 0.2267, 0.7199, 0.6313, 0.9275],
         [0.7673, 0.1414, 0.6721, 0.2191, 0.7041, 0.9413, 0.4436, 0.6410,
          0.9704, 0.8997, 0.8745, 0.3538, 0.3665, 0.6780, 0.9737, 0.9318],
         [0.0051, 0.3077, 0.4717, 0.7149, 0.8349, 0.2188, 0.5688, 0.5073,
          0.4032, 0.7420, 0.5170, 0.5111, 0.0541, 0.1154, 0.6693, 0.0838],
         [0.1339, 0.6915, 0.9916, 0.4239, 0.4172, 0.9841, 0.2820, 0.0583,
          0.9355, 0.7958, 0.1894

In [19]:
image_tensor.shape

torch.Size([3, 9, 16])

Можем попробовать поменять размер тензора, например, [вытянуть его в вектор](https://pytorch.org/docs/stable/generated/torch.ravel.html):

In [20]:
image_tensor.ravel()

tensor([0.4864, 0.1345, 0.0985, 0.4910, 0.5191, 0.5402, 0.9424, 0.2336, 0.8935,
        0.6876, 0.0369, 0.8935, 0.2823, 0.6079, 0.8713, 0.8276, 0.4903, 0.8007,
        0.8147, 0.8142, 0.5783, 0.7870, 0.6481, 0.8210, 0.4620, 0.7126, 0.3961,
        0.2064, 0.6209, 0.7252, 0.0013, 0.8203, 0.6782, 0.4787, 0.7272, 0.7661,
        0.1971, 0.1114, 0.1537, 0.6295, 0.8935, 0.4559, 0.6645, 0.7148, 0.9582,
        0.4237, 0.3216, 0.0474, 0.9153, 0.7582, 0.9807, 0.7008, 0.4528, 0.5591,
        0.9300, 0.8818, 0.0474, 0.9770, 0.0913, 0.7871, 0.2267, 0.7199, 0.6313,
        0.9275, 0.7673, 0.1414, 0.6721, 0.2191, 0.7041, 0.9413, 0.4436, 0.6410,
        0.9704, 0.8997, 0.8745, 0.3538, 0.3665, 0.6780, 0.9737, 0.9318, 0.0051,
        0.3077, 0.4717, 0.7149, 0.8349, 0.2188, 0.5688, 0.5073, 0.4032, 0.7420,
        0.5170, 0.5111, 0.0541, 0.1154, 0.6693, 0.0838, 0.1339, 0.6915, 0.9916,
        0.4239, 0.4172, 0.9841, 0.2820, 0.0583, 0.9355, 0.7958, 0.1894, 0.4010,
        0.1479, 0.5418, 0.3030, 0.3910, 

In [21]:
image_tensor.ravel().shape

torch.Size([432])

In [22]:
h * w * c

432

Посчитаем количество элементов в тензоре с помощью [специальной функции](https://pytorch.org/docs/stable/generated/torch.numel.html):

In [23]:
image_tensor.numel()

432

In [24]:
h = 2
w = 3
c = 3

shape = (c, h, w)

image_tensor = torch.rand(shape)

image_tensor

tensor([[[0.4597, 0.1813, 0.6826],
         [0.1960, 0.6341, 0.8384]],

        [[0.3921, 0.1819, 0.0802],
         [0.2908, 0.9219, 0.7294]],

        [[0.7496, 0.6092, 0.0628],
         [0.2635, 0.2133, 0.8289]]])

Попробуем поменять размер с помощью функции [reshape](https://pytorch.org/docs/stable/generated/torch.reshape.html#torch.reshape):

In [25]:
image_tensor.reshape(c, h * w)

tensor([[0.4597, 0.1813, 0.6826, 0.1960, 0.6341, 0.8384],
        [0.3921, 0.1819, 0.0802, 0.2908, 0.9219, 0.7294],
        [0.7496, 0.6092, 0.0628, 0.2635, 0.2133, 0.8289]])

Попробуем собрать из нескольких тензоров один большой:

[torch.cat](https://pytorch.org/docs/stable/generated/torch.cat.html#torch.cat)

In [26]:
x = torch.randn(2, 3)

In [27]:
x

tensor([[-0.3412,  1.6440, -0.8311],
        [-0.1244, -0.8518, -1.6014]])

In [28]:
torch.cat((x, x, x), dim=0)

tensor([[-0.3412,  1.6440, -0.8311],
        [-0.1244, -0.8518, -1.6014],
        [-0.3412,  1.6440, -0.8311],
        [-0.1244, -0.8518, -1.6014],
        [-0.3412,  1.6440, -0.8311],
        [-0.1244, -0.8518, -1.6014]])

In [38]:
torch.stack((x, x, x, x), dim=0).shape

torch.Size([4, 2, 3])

In [42]:
x.unsqueeze_(0).shape

torch.Size([1, 1, 2, 3])

In [30]:
x = torch.randn(3, 3)
y = torch.randn(5, 3)
z = torch.randn(1, 3)

for tensor in [x, y, z]:
    print(tensor)

torch.cat((x, y, z), dim=0)

tensor([[ 0.1528, -0.7060, -0.6123],
        [ 0.0732, -0.1394,  0.1681],
        [-0.0177, -0.2970, -1.0488]])
tensor([[ 0.7031, -1.7171,  0.7060],
        [-0.1856, -0.0150,  1.4701],
        [ 0.1259, -1.4528,  0.2876],
        [ 1.2115, -0.5579, -0.1800],
        [-0.3268,  0.8831,  0.0295]])
tensor([[ 0.8707,  0.1631, -0.7366]])


tensor([[ 0.1528, -0.7060, -0.6123],
        [ 0.0732, -0.1394,  0.1681],
        [-0.0177, -0.2970, -1.0488],
        [ 0.7031, -1.7171,  0.7060],
        [-0.1856, -0.0150,  1.4701],
        [ 0.1259, -1.4528,  0.2876],
        [ 1.2115, -0.5579, -0.1800],
        [-0.3268,  0.8831,  0.0295],
        [ 0.8707,  0.1631, -0.7366]])

In [34]:
x = torch.randn(2, 3)
y = torch.randn(2, 5)
z = torch.randn(2, 1)

for tensor in [x, y, z]:
    print(tensor)

torch.stack((x, y, z), dim=1)

tensor([[ 2.1165,  1.0163, -0.0865],
        [-0.3076, -0.9481,  0.8369]])
tensor([[-0.6109,  1.0249,  1.0674, -0.9915,  0.5518],
        [-1.8560,  1.6864, -0.3360, -2.1264,  1.3142]])
tensor([[-0.3127],
        [-0.2211]])


RuntimeError: stack expects each tensor to be equal size, but got [2, 3] at entry 0 and [2, 5] at entry 1

Теперь добавим дополнительную ось:

[torch.unsqueeze](https://pytorch.org/docs/stable/generated/torch.unsqueeze.html)

In [32]:
x = torch.rand(2, 3)

print(x)
print()
print(x.unsqueeze(0), x.unsqueeze(0).shape)
print()
print(x.unsqueeze(1), x.unsqueeze(1).shape)
print()
print(x.unsqueeze(2), x.unsqueeze(2).shape)

tensor([[0.3201, 0.5346, 0.4294],
        [0.7934, 0.3562, 0.2969]])

tensor([[[0.3201, 0.5346, 0.4294],
         [0.7934, 0.3562, 0.2969]]]) torch.Size([1, 2, 3])

tensor([[[0.3201, 0.5346, 0.4294]],

        [[0.7934, 0.3562, 0.2969]]]) torch.Size([2, 1, 3])

tensor([[[0.3201],
         [0.5346],
         [0.4294]],

        [[0.7934],
         [0.3562],
         [0.2969]]]) torch.Size([2, 3, 1])


Уберем лишние оси (где размер единичка):

In [45]:
x = torch.rand(1, 2, 1, 3)

print(x)
print()
print(x.squeeze(), x.squeeze().shape)
print()
print(x.squeeze(1), x.squeeze(1).shape)

tensor([[[[0.0557, 0.0066, 0.7910]],

         [[0.4676, 0.4237, 0.3102]]]])

tensor([[0.0557, 0.0066, 0.7910],
        [0.4676, 0.4237, 0.3102]]) torch.Size([2, 3])

tensor([[[[0.0557, 0.0066, 0.7910]],

         [[0.4676, 0.4237, 0.3102]]]]) torch.Size([1, 2, 1, 3])


Теперь поговорим про типы данных в тензорах. По умолчанию в тензорах лежат числа в torch.float32 для вещественных и torch.int64 для целочисленных.

In [46]:
tensor = torch.tensor([1.5, 2.2, 3.7, 4.9])

tensor

tensor([1.5000, 2.2000, 3.7000, 4.9000])

In [47]:
tensor.dtype

torch.float32

In [48]:
tensor = torch.tensor([1.5, 2.2, 3.7, 4.9], dtype=torch.float16)

tensor

tensor([1.5000, 2.1992, 3.6992, 4.8984], dtype=torch.float16)

In [49]:
tensor = torch.tensor([1.5, 2.2, 3.7, 4.9], dtype=torch.float64)

tensor

tensor([1.5000, 2.2000, 3.7000, 4.9000], dtype=torch.float64)

In [50]:
tensor = torch.tensor([15, 22, 37, 49])

tensor

tensor([15, 22, 37, 49])

In [51]:
tensor.dtype

torch.int64

In [52]:
tensor = torch.tensor([15, 22, 37, 49], dtype=torch.int32)

tensor

tensor([15, 22, 37, 49], dtype=torch.int32)

In [53]:
tensor = torch.tensor([15, 22, 37, 49], dtype=torch.int16)

tensor

tensor([15, 22, 37, 49], dtype=torch.int16)

Размещение тензора на GPU:

In [54]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
NVIDIA GeForce RTX 4090


In [ ]:
! nvidia-smi

In [55]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

cuda:0


In [56]:
tensor = torch.tensor([15, 22, 37, 49], device=device)

tensor

tensor([15, 22, 37, 49], device='cuda:0')

In [57]:
tensor = torch.tensor([15, 22, 37, 49])

print(tensor)

tensor = tensor.to(device)

tensor

tensor([15, 22, 37, 49])


tensor([15, 22, 37, 49], device='cuda:0')

In [58]:
tensor.to(torch.int32)

tensor([15, 22, 37, 49], device='cuda:0', dtype=torch.int32)

In [59]:
tensor = tensor.cpu()

tensor

tensor([15, 22, 37, 49])

In [60]:
tensor.cuda()

tensor([15, 22, 37, 49], device='cuda:0')

In [61]:
a = torch.rand(2, 3)
b = torch.rand(2, 3)

a + b

tensor([[1.4667, 0.9306, 1.3612],
        [1.0351, 0.9423, 1.1146]])

In [62]:
a = a.to(device)

a + b

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [63]:
b = b.to(device)

a + b

tensor([[1.4667, 0.9306, 1.3612],
        [1.0351, 0.9423, 1.1146]], device='cuda:0')

### Операции с тензорами

Большая часть операций с тензорами хорошо описана в их [документации](https://pytorch.org/docs/stable/torch.html), разберем основные:

In [64]:
a = torch.rand(2, 3)
b = torch.rand(2, 3)

a, b

(tensor([[0.8188, 0.6701, 0.4402],
         [0.7288, 0.2568, 0.7191]]),
 tensor([[0.2241, 0.6612, 0.6350],
         [0.8124, 0.5353, 0.0116]]))

In [65]:
# поэлементные

print(a + b)

print()

print(torch.add(a, b))

print()

print(a.add(b))

tensor([[1.0429, 1.3313, 1.0753],
        [1.5412, 0.7921, 0.7307]])

tensor([[1.0429, 1.3313, 1.0753],
        [1.5412, 0.7921, 0.7307]])

tensor([[1.0429, 1.3313, 1.0753],
        [1.5412, 0.7921, 0.7307]])


In [66]:
print(a - b)

print()

print(torch.sub(a, b))

print()

print(a.sub(b))

tensor([[ 0.5947,  0.0089, -0.1948],
        [-0.0836, -0.2785,  0.7075]])

tensor([[ 0.5947,  0.0089, -0.1948],
        [-0.0836, -0.2785,  0.7075]])

tensor([[ 0.5947,  0.0089, -0.1948],
        [-0.0836, -0.2785,  0.7075]])


In [67]:
print(a * b)

print()

print(torch.mul(a, b))

print()

print(a.mul(b))

tensor([[0.1835, 0.4430, 0.2796],
        [0.5921, 0.1375, 0.0083]])

tensor([[0.1835, 0.4430, 0.2796],
        [0.5921, 0.1375, 0.0083]])

tensor([[0.1835, 0.4430, 0.2796],
        [0.5921, 0.1375, 0.0083]])


In [68]:
print(a / b)

print()

print(torch.div(a, b))

print()

print(a.div(b))

tensor([[ 3.6534,  1.0134,  0.6932],
        [ 0.8971,  0.4798, 61.9581]])

tensor([[ 3.6534,  1.0134,  0.6932],
        [ 0.8971,  0.4798, 61.9581]])

tensor([[ 3.6534,  1.0134,  0.6932],
        [ 0.8971,  0.4798, 61.9581]])


In [69]:
a = torch.rand(2, 3)
b = torch.rand(3, 4)
c = torch.rand(5, 5)

a, b, c

(tensor([[0.5259, 0.9298, 0.9535],
         [0.8992, 0.3379, 0.9277]]),
 tensor([[0.6497, 0.7538, 0.6800, 0.4877],
         [0.3039, 0.3895, 0.0844, 0.0290],
         [0.4299, 0.7565, 0.3106, 0.4319]]),
 tensor([[0.2364, 0.1823, 0.2378, 0.3356, 0.7894],
         [0.1962, 0.3696, 0.2490, 0.3883, 0.8007],
         [0.1639, 0.3019, 0.6598, 0.9254, 0.7836],
         [0.2299, 0.7600, 0.6759, 0.2383, 0.0970],
         [0.7522, 0.0800, 0.5078, 0.3682, 0.0312]]))

In [70]:
# матричные операции

print(a @ b, (a @ b).shape)

print()

print(torch.matmul(a, b), torch.matmul(a, b).shape)

print()

print(c.trace())

print()

print(c.exp())

tensor([[1.0342, 1.4800, 0.7322, 0.6953],
        [1.0857, 1.5113, 0.9281, 0.8490]]) torch.Size([2, 4])

tensor([[1.0342, 1.4800, 0.7322, 0.6953],
        [1.0857, 1.5113, 0.9281, 0.8490]]) torch.Size([2, 4])

tensor(1.5353)

tensor([[1.2667, 1.2000, 1.2684, 1.3987, 2.2020],
        [1.2168, 1.4472, 1.2827, 1.4745, 2.2270],
        [1.1781, 1.3525, 1.9345, 2.5228, 2.1894],
        [1.2585, 2.1384, 1.9658, 1.2690, 1.1019],
        [2.1216, 1.0833, 1.6616, 1.4452, 1.0317]])


### [Автоматическое дифференцирование](https://pytorch.org/docs/stable/notes/autograd.html)

In [71]:
x = torch.rand(5)

x

tensor([0.2449, 0.8752, 0.7939, 0.2759, 0.9019])

In [72]:
w = torch.rand(3, 5, requires_grad=True)

w

tensor([[0.1584, 0.7771, 0.9942, 0.3884, 0.1884],
        [0.2157, 0.0236, 0.8297, 0.9158, 0.3040],
        [0.7488, 0.3383, 0.6428, 0.6155, 0.0773]], requires_grad=True)

In [73]:
print(w.grad)

None


In [ ]:
first_z = torch.empty(3)

first_z

In [ ]:
for i in range(3):
    first_z[i] = torch.sum(w[i] * x)

first_z

In [74]:
z = torch.matmul(x, w.t())

z

tensor([1.7853, 1.2590, 1.2293], grad_fn=<SqueezeBackward4>)

In [75]:
v = torch.rand(3, requires_grad=True)

v

tensor([0.8576, 0.9782, 0.5280], requires_grad=True)

In [76]:
print(v.grad)

None


In [77]:
y = torch.sum(z * v)

y

tensor(3.4118, grad_fn=<SumBackward0>)

In [78]:
y.item()

3.411832332611084

In [79]:
loss = torch.mean((y - 2) ** 2)

In [80]:
loss

tensor(1.9933, grad_fn=<MeanBackward0>)

In [83]:
print(f'{x.grad=}\n')
print(f'{w.grad=}\n')
print(f'{z.grad=}\n')
print(f'{v.grad=}\n')

x.grad=None

w.grad=None

z.grad=None

v.grad=None



/tmp/ipykernel_44/162867685.py:3: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:486.)
  print(f'{z.grad=}\n')


In [84]:
loss.backward()

In [85]:
print(f'{x.grad=}\n')
print(f'{w.grad=}\n')
print(f'{z.grad=}\n')
print(f'{v.grad=}\n')

x.grad=None

w.grad=tensor([[0.5932, 2.1195, 1.9226, 0.6681, 2.1841],
        [0.6765, 2.4174, 2.1928, 0.7620, 2.4910],
        [0.3652, 1.3050, 1.1837, 0.4114, 1.3447]])

z.grad=None

v.grad=tensor([5.0411, 3.5550, 3.4712])



/tmp/ipykernel_44/162867685.py:3: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:486.)
  print(f'{z.grad=}\n')


In [86]:
a = torch.rand(1, requires_grad=True)
b = torch.rand(1, requires_grad=True)

a, b

(tensor([0.8304], requires_grad=True), tensor([0.1626], requires_grad=True))

In [87]:
loss = (a - b)

loss

tensor([0.6679], grad_fn=<SubBackward0>)

In [88]:
print(f'{a.grad=}\n')
print(f'{b.grad=}\n')

a.grad=None

b.grad=None



In [89]:
loss.backward()

In [92]:
print(f'{a.grad=}\n')  # 1
print(f'{b.grad=}\n')  # -1

a.grad=tensor([0.])

b.grad=tensor([0.])



In [91]:
a.grad.zero_()
b.grad.zero_()

tensor([0.])

In [93]:
loss = (a - b) ** 2

loss

tensor([0.4460], grad_fn=<PowBackward0>)

In [94]:
print(f'{a.grad=}\n')
print(f'{b.grad=}\n')

a.grad=tensor([0.])

b.grad=tensor([0.])



In [95]:
loss.backward()

In [96]:
print(f'{a.grad=}\n')  # 2 * (a - b)
print(f'{b.grad=}\n')  # -2 * (a - b)

a.grad=tensor([1.3357])

b.grad=tensor([-1.3357])



In [97]:
2 * (a - b)

tensor([1.3357], grad_fn=<MulBackward0>)

In [ ]:
a = torch.rand(3, 5, requires_grad=True)
b = torch.rand(3, 5, requires_grad=True)

a, b

In [ ]:
loss = torch.mean(a * b)

loss

In [ ]:
print(f'{a.grad=}\n')
print(f'{b.grad=}\n')

In [ ]:
loss.backward()

In [ ]:
print(f'{a.grad=}\n')  # b / (3 * 5)
print(f'{b.grad=}\n')  # a / (3 * 5)

In [ ]:
a / 15

In [ ]:
b / 15

In [98]:
a = torch.rand(3, 5, requires_grad=True)

print(f'{a=}\n')

loss1 = torch.sum(a ** 2) # 2a
loss2 = torch.sum(a) # 1

print(f'{a.grad=}\n')

loss1.backward()

print(f'{a.grad=}\n')

loss2.backward()

print(f'{a.grad=}\n')

a=tensor([[0.0913, 0.9977, 0.1392, 0.3450, 0.1168],
        [0.7531, 0.6068, 0.3647, 0.8633, 0.3479],
        [0.0462, 0.6566, 0.2915, 0.5604, 0.9624]], requires_grad=True)

a.grad=None

a.grad=tensor([[0.1826, 1.9954, 0.2783, 0.6899, 0.2336],
        [1.5063, 1.2137, 0.7294, 1.7265, 0.6959],
        [0.0925, 1.3132, 0.5829, 1.1207, 1.9247]])

a.grad=tensor([[1.1826, 2.9954, 1.2783, 1.6899, 1.2336],
        [2.5063, 2.2137, 1.7294, 2.7265, 1.6959],
        [1.0925, 2.3132, 1.5829, 2.1207, 2.9247]])



In [ ]:
print(f'{2*a=}\n')
print(f'{2*a+1=}')

In [ ]:
a = torch.rand(3, 5, requires_grad=True)
b = torch.rand(3, 5, requires_grad=False)

a, b

In [ ]:
loss = torch.sum(a - b)

loss

In [ ]:
print(f'{a.grad=}\n')
print(f'{b.grad=}\n')

In [ ]:
loss.backward()

In [ ]:
print(f'{a.grad=}\n')  # all ones
print(f'{b.grad=}\n')  # None

In [99]:
a = torch.rand(3, 5, requires_grad=True)
b = torch.rand(3, 5, requires_grad=True)

a, b

(tensor([[0.1631, 0.7200, 0.7540, 0.9201, 0.1146],
         [0.3966, 0.8201, 0.2972, 0.3315, 0.2346],
         [0.8444, 0.2682, 0.8428, 0.7369, 0.8682]], requires_grad=True),
 tensor([[0.5493, 0.4100, 0.6125, 0.1204, 0.3800],
         [0.6869, 0.0135, 0.8407, 0.2881, 0.9219],
         [0.5137, 0.4388, 0.1657, 0.4579, 0.7925]], requires_grad=True))

In [100]:
with torch.no_grad():
    loss = torch.sum(a - b)

loss

tensor(1.1200)

In [101]:
loss.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [102]:
a = torch.rand(3, 5, requires_grad=True)
b = torch.rand(3, 5, requires_grad=True)

a, b

(tensor([[0.4820, 0.0068, 0.5513, 0.5632, 0.5633],
         [0.8886, 0.9419, 0.0881, 0.0107, 0.3856],
         [0.4950, 0.7354, 0.0205, 0.1851, 0.9359]], requires_grad=True),
 tensor([[0.7496, 0.0309, 0.1084, 0.8919, 0.1318],
         [0.6762, 0.1406, 0.4177, 0.5294, 0.3781],
         [0.7381, 0.2424, 0.3491, 0.8402, 0.0801]], requires_grad=True))

In [103]:
with torch.inference_mode():
    loss = torch.sum(a - b)

loss

tensor(0.5486)

In [104]:
loss.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [105]:
with torch.no_grad():
    a = torch.rand(3, 5, requires_grad=True)
    b = torch.rand(3, 5, requires_grad=True)
    
    loss = torch.sum(a + b)
    
    print(f'{loss=}')
    
    loss.backward()

loss=tensor(16.6919)


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [106]:
loss2 = torch.sum(a + b)

loss2

tensor(16.6919, grad_fn=<SumBackward0>)

In [107]:
print(f'{a.grad=}\n')
print(f'{b.grad=}\n')

a.grad=None

b.grad=None



In [108]:
loss2.backward()

In [109]:
print(f'{a.grad=}\n')
print(f'{b.grad=}\n')

a.grad=tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

b.grad=tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])



In [110]:
with torch.inference_mode():
    a = torch.rand(3, 5, requires_grad=True)
    b = torch.rand(3, 5, requires_grad=True)
    
    loss = torch.sum(a + b)
    
    print(f'{loss=}')
    
    loss.backward()

loss=tensor(12.6187)


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [111]:
loss2 = torch.sum(a + b)

loss2

tensor(12.6187)

In [112]:
@torch.no_grad()
def foo():
    a = torch.rand(3, 5, requires_grad=True)
    b = torch.rand(3, 5, requires_grad=True)
    
    loss = torch.mean(a + b)
    
    print(f'{loss=}')
    
    return a, b

In [113]:
a, b = foo()

loss=tensor(0.9662)


In [114]:
torch.mean(a - b)

tensor(-0.0883, grad_fn=<MeanBackward0>)

In [115]:
@torch.inference_mode()
def foo():
    a = torch.rand(3, 5, requires_grad=True)
    b = torch.rand(3, 5, requires_grad=True)
    
    loss = torch.mean(a + b)
    
    print(f'{loss=}')
    
    return a, b

In [116]:
a, b = foo()

loss=tensor(0.9661)


In [117]:
torch.mean(a - b)

tensor(0.0121)

In [118]:
a.requires_grad = False

## Полносвязные слои и функции активации в `PyTorch`

In [119]:
from torch import nn

### Полносвязный слой

>$y_j = \sum\limits_{i=1}^{n}x_iw_{ji} + b_j$


In [120]:
layer = nn.Linear(in_features=5, out_features=3)

In [121]:
layer

Linear(in_features=5, out_features=3, bias=True)

In [122]:
layer.weight

Parameter containing:
tensor([[ 0.2526, -0.0183,  0.2870, -0.4051,  0.0200],
        [ 0.2125, -0.4402,  0.3310,  0.0280, -0.0666],
        [-0.3902,  0.1472,  0.1944,  0.3951, -0.4383]], requires_grad=True)

In [123]:
layer.weight.shape

torch.Size([3, 5])

In [124]:
layer.bias

Parameter containing:
tensor([-0.3035,  0.3197, -0.3720], requires_grad=True)

In [125]:
layer = nn.Linear(in_features=5, out_features=3, bias=False)

In [126]:
layer.bias

In [127]:
layer.__call__

<bound method Module._call_impl of Linear(in_features=5, out_features=3, bias=False)>

In [128]:
x = torch.randn(5)

print(layer(x))

tensor([ 0.4626, -0.0048,  0.4168], grad_fn=<SqueezeBackward4>)


In [129]:
layer1 = nn.Linear(in_features=5, out_features=3)
layer2 = nn.Linear(in_features=3, out_features=1)

layer2(layer1(x))

tensor([0.1300], grad_fn=<AddBackward0>)

### Функции активации

> Сигмоида $f(x) = \dfrac{1}{1 + e^{-x}}$

In [130]:
activation = nn.Sigmoid()

In [131]:
x = torch.randn(5)

print(x)

print(activation(x))

tensor([-0.1609, -1.0230,  0.5188,  0.1823,  0.9216])
tensor([0.4599, 0.2644, 0.6269, 0.5455, 0.7154])


> ReLU $f(x) = \max(0, x)$

In [132]:
activation = nn.ReLU()

In [133]:
x = torch.randn(5)

print(x)

print(activation(x))

tensor([ 0.2526,  0.8336,  1.7116,  0.1765, -1.2696])
tensor([0.2526, 0.8336, 1.7116, 0.1765, 0.0000])


> Leaky ReLU $f(x) = \max(0, x) + \alpha \min(0, x)$

In [134]:
activation = nn.LeakyReLU(negative_slope=0.001)

In [135]:
x = torch.randn(5)

print(x)

print(activation(x))

tensor([ 1.2277, -0.3938, -0.4653,  0.4705,  0.7487])
tensor([ 1.2277e+00, -3.9383e-04, -4.6527e-04,  4.7049e-01,  7.4869e-01])


In [136]:
layer1 = nn.Linear(in_features=5, out_features=3)
activation = nn.LeakyReLU(negative_slope=0.001)
layer2 = nn.Linear(in_features=3, out_features=1)

layer2(activation(layer1(x)))

tensor([0.4624], grad_fn=<AddBackward0>)

## Градиентный спуск своими руками

In [ ]:
n_features = 2
n_objects = 300

torch.manual_seed(0)

w_true = torch.randn(n_features)
b_true = torch.randn(1)

x = (torch.rand(n_objects, n_features) - 0.5) * 10 * (torch.arange(n_features) * 2 + 1)
y = torch.matmul(x, w_true) + torch.randn(n_objects) + b_true

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
n_steps = 200
step_size = 1e-2

In [ ]:
w = torch.rand(n_features, requires_grad=True)
b = torch.rand(1, requires_grad=True)

for i in range(n_steps):
    y_pred = torch.matmul(x, w) + b

    mse = torch.mean((y_pred - y) ** 2)

    if i < 20 or i % 10 == 0:
        print(f'MSE на шаге {i + 1} {mse.item():.5f}')

    mse.backward()
    
#     print(f'{w.grad=}\n')
#     print(f'{b.grad=}\n')

    with torch.no_grad():
        w -= w.grad * step_size
        b -= b.grad * step_size

    w.grad.zero_()
    b.grad.zero_()

In [ ]:
layer = nn.Linear(in_features=n_features, out_features=1)


for i in range(n_steps):
    y_pred = layer(x)

    mse = torch.mean((y_pred - y) ** 2)
    
    if i < 20 or i % 10 == 0:
        print(f'MSE на шаге {i + 1} {mse.item():.5f}')

    mse.backward()

    with torch.no_grad():
        layer.weight -= layer.weight.grad * step_size
        layer.bias -= layer.bias.grad * step_size

#     layer.weight.grad.zero_()
#     layer.bias.grad.zero_()
    
    layer.zero_grad()

In [ ]:
layer(x)

In [ ]:
layer(x).shape

In [ ]:
y.shape

In [ ]:
(layer(x) - y).shape

In [ ]:
layer(x).ravel()

In [ ]:
(layer(x).ravel() - y).shape

In [ ]:
layer = nn.Linear(in_features=n_features, out_features=1)

for i in range(n_steps):
    y_pred = layer(x).ravel()

    mse = torch.mean((y_pred - y) ** 2)
    
    if i < 20 or i % 10 == 0:
        print(f'MSE на шаге {i + 1} {mse.item():.5f}')

    mse.backward()

    with torch.no_grad():
        layer.weight -= layer.weight.grad * step_size
        layer.bias -= layer.bias.grad * step_size

    layer.zero_grad()

In [ ]:
n_features = 5
n_objects = 300

torch.manual_seed(0)

w_true = torch.randn(n_features)

x = (torch.rand(n_objects, n_features) - 0.5) * 10 * (torch.arange(n_features) * 2 + 1)
y = torch.matmul(x, w_true) + torch.randn(n_objects)

In [ ]:
x.shape, y.shape

In [ ]:
n_steps = 500
step_size = 1e-3

In [ ]:
layer = nn.Linear(in_features=n_features, out_features=1)

for i in range(n_steps):
    y_pred = layer(x).ravel()

    mse = torch.mean((y_pred - y) ** 2)
    
    if i < 20 or i % 50 == 0:
        print(f'MSE на шаге {i + 1} {mse.item():.5f}')

    mse.backward()

    with torch.no_grad():
        layer.weight -= layer.weight.grad * step_size
        layer.bias -= layer.bias.grad * step_size

    layer.zero_grad()

In [ ]:
n_steps = 1000
step_size = 3e-4

In [ ]:
layer1 = nn.Linear(in_features=n_features, out_features=3)
layer2 = nn.Linear(in_features=3, out_features=1)
activation = nn.ReLU()


for i in range(n_steps):
    y_pred = layer2(activation(layer1(x))).ravel()

    mse = torch.mean((y_pred - y) ** 2)

    if i < 20 or i % 50 == 0:
        print(f'MSE на шаге {i + 1} {mse.item():.5f}')

    mse.backward()

    with torch.no_grad():
        layer1.weight -= layer1.weight.grad * step_size
        layer1.bias -= layer1.bias.grad * step_size
        layer2.weight -= layer2.weight.grad * step_size
        layer2.bias -= layer2.bias.grad * step_size

    layer1.zero_grad()
    layer2.zero_grad()